Существует несколько библиотек для автоматического машинного обучения (AutoML) в Keras. Некоторые из них:

*AutoKeras*: это библиотека AutoML, разработанная для Keras, которая автоматически определяет архитектуру нейронной сети и настраивает ее гиперпараметры. AutoKeras также предоставляет механизмы автоматического извлечения признаков и визуализации результатов.

*Keras Tuner*: это библиотека для настройки гиперпараметров моделей Keras. Keras Tuner предоставляет механизмы для выбора лучшей комбинации гиперпараметров, включая случайный поиск, поиск сетки и байесовский поиск.

*Talos*: это библиотека AutoML для Keras, которая использует байесовскую оптимизацию для выбора наилучших гиперпараметров модели. Talos также предоставляет механизмы автоматического создания отчетов и визуализации результатов.

*Hyperas*: это библиотека для автоматической настройки гиперпараметров Keras с использованием поиска по сетке. Hyperas позволяет определить границы для гиперпараметров, которые нужно настроить, и выполняет автоматическую настройку на основе заданной функции оценки.

Это не полный список, но эти библиотеки предоставляют множество возможностей для автоматической настройки моделей Keras.

# Подготовка и загрузка датасета с изображениями водителей

In [ ]:
import gdown                                     # Подключение модуля для загрузки данных из облака
import numpy as np                               # Библиотека работы с массивами
import matplotlib.pyplot as plt                  # Для отрисовки графиков
%matplotlib inline
from PIL import Image                            # Для отрисовки изображений
import random as random                          # Генератор рандомных чисел
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam                # Оптимизатор Adam
from tensorflow.keras.models import Sequential   # Сеть прямого распространения
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras import utils               # Используем для to_categorical
from tensorflow.keras.preprocessing import image # Для отрисовки изображений
from google.colab import files                   # Для загрузки своей картинки
import os
import time
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# указываем количество категорий правильных ответов
category_count = 10
# Загрузка zip-архива с датасетом из облака на диск виртуальной машины colab
gdown.download('https://storage.yandexcloud.net/aiueducation/marketing/datasets/reality.zip', None, quiet=True)

'reality.zip'

In [ ]:
# Разархивация датасета в директорию 'content/cars'
!unzip -qo "reality.zip" -d /content/reality

# Папка с папками картинок, рассортированных по категориям
IMAGE_PATH = '/content/reality/'

In [ ]:
# Список с названиями категорий
os.listdir(IMAGE_PATH)

['safe_driving',
 'drinking',
 'texting_right',
 'reaching_behind',
 'operating_the_radio',
 'texting_left',
 'talking_on_the_phone_right',
 'hair_and_makeup',
 'talking_to_passenger',
 'talking_on_the_phone_left']

In [ ]:
# Определение списка имен классов
CLASS_LIST = sorted(os.listdir(IMAGE_PATH))

# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)

# Проверка результата
print(f'Количество классов: {CLASS_COUNT}, метки классов: {CLASS_LIST}')

Количество классов: 10, метки классов: ['drinking', 'hair_and_makeup', 'operating_the_radio', 'reaching_behind', 'safe_driving', 'talking_on_the_phone_left', 'talking_on_the_phone_right', 'talking_to_passenger', 'texting_left', 'texting_right']


In [ ]:
data_files = []                           # Cписок путей к файлам картинок
data_labels = []                          # Список меток классов, соответствующих файлам

for class_label in range(CLASS_COUNT):    # Для всех классов по порядку номеров (их меток)
    class_name = CLASS_LIST[class_label]  # Выборка имени класса из списка имен
    class_path = IMAGE_PATH + class_name  # Формирование полного пути к папке с изображениями класса
    class_files = os.listdir(class_path)  # Получение списка имен файлов с изображениями текущего класса
    print(f'Размер класса {class_name} составляет {len(class_files)} изображений')

    # Добавление к общему списку всех файлов класса с добавлением родительского пути
    data_files += [f'{class_path}/{file_name}' for file_name in class_files]

    # Добавление к общему списку меток текущего класса - их ровно столько, сколько файлов в классе
    data_labels += [class_label] * len(class_files)

print('Общий размер базы для обучения:', len(data_labels))

Размер класса drinking составляет 300 изображений
Размер класса hair_and_makeup составляет 300 изображений
Размер класса operating_the_radio составляет 300 изображений
Размер класса reaching_behind составляет 300 изображений
Размер класса safe_driving составляет 300 изображений
Размер класса talking_on_the_phone_left составляет 300 изображений
Размер класса talking_on_the_phone_right составляет 300 изображений
Размер класса talking_to_passenger составляет 300 изображений
Размер класса texting_left составляет 300 изображений
Размер класса texting_right составляет 300 изображений
Общий размер базы для обучения: 3000


In [ ]:
IMG_WIDTH = 64                           # Ширина изображения
IMG_HEIGHT = 64                           # Высота изображения

In [ ]:
data_images = []                          # Пустой список для данных изображений

for file_name in data_files:
    # Открытие и смена размера изображения
    img = Image.open(file_name).resize((IMG_WIDTH, IMG_HEIGHT)) 
    img_np = np.array(img)                # Перевод в numpy-массив
    data_images.append(img_np)            # Добавление изображения в виде numpy-массива к общему списку

x_data = np.array(data_images)            # Перевод общего списка изображений в numpy-массив
y_data = np.array(data_labels)            # Перевод общего списка меток класса в numpy-массив

print(f'В массив собрано {len(data_images)} фотографий следующей формы: {img_np.shape}')
print(f'Общий массив данных изображений следующей формы: {x_data.shape}')
print(f'Общий массив меток классов следующей формы: {y_data.shape}')

В массив собрано 3000 фотографий следующей формы: (64, 64, 3)
Общий массив данных изображений следующей формы: (3000, 64, 64, 3)
Общий массив меток классов следующей формы: (3000,)


In [ ]:
x_data.shape

(3000, 64, 64, 3)

In [ ]:
# Нормированние массива изображений
x_data = x_data / 255.

In [ ]:
# Делим данные на обучающую и проверочную выборки
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.05, random_state=42)

In [ ]:
# Задаем batch_size
batch_size = 128

# Загружаем названия классов из набора данных по водителям:
classes=['drinking ', 'hair_and_makeup ', 'operating_the_radio ', 'reaching_behind ', 'safe_driving ', 'talking_on_the_phone_left  ', 'talking_on_the_phone_right  ', 'talking_to_passenger  ', 'texting_left  ', 'texting_right ']

# Превращаем выходы сетей в формат  one hot encoding
y_train = utils.to_categorical(y_train, category_count)

pic_shapes = x_train[0].shape
IMG_WIDTH = pic_shapes[0]
IMG_HEIGHT = pic_shapes[1]
num_channels = pic_shapes[2]

# Делаем решейп
x_train = x_train.reshape(x_train.shape[0], IMG_WIDTH, IMG_HEIGHT, num_channels)
x_val = x_val.reshape(x_val.shape[0],IMG_WIDTH, IMG_HEIGHT, num_channels)

In [ ]:
y_val = utils.to_categorical(y_val, category_count)

In [ ]:
print(x_train.shape)
print(x_val.shape)

(2850, 64, 64, 3)
(150, 64, 64, 3)


In [ ]:
print(y_train.shape)
print(y_val.shape)

(2850, 10)
(150, 10)


# AutoKeras

Auto-Keras (AutoML for deep learning) - это библиотека, предоставляющая возможность автоматизировать процесс создания глубоких нейронных сетей без участия человека. Она позволяет быстро и эффективно находить лучшие гиперпараметры модели на основе различных алгоритмов оптимизации и случайного поиска. Библиотека AutoKeras была выпущена в 2018 году.

Библиотека может работать с изображениями, текстами и табличными данными.

Основная идея Auto-Keras заключается в том, чтобы автоматически находить наиболее эффективные архитектуры глубоких нейронных сетей на основе процесса обучения с подкреплением (reinforcement learning). В процессе работы Auto-Keras генерирует случайные архитектуры сетей и оценивает их производительность на основе метрик, заданных пользователем. Затем происходит отбор лучших моделей и дальнейшая оптимизация гиперпараметров, таких как количество слоев, их тип, количество нейронов в каждом слое, функции активации и т.д.

Auto-Keras использует набор примитивов, позволяющих генерировать различные архитектуры нейронных сетей, основанных на классических методах, таких как сверточные нейронные сети (Convolutional Neural Networks, CNN) и рекуррентные нейронные сети (Recurrent Neural Networks, RNN), а также на более современных методах, таких как ResNet и DenseNet. При этом процесс генерации новых моделей может занимать много времени и ресурсов.

Auto-Keras поддерживает использование как CPU, так и GPU, что позволяет быстрее обрабатывать большие объемы данных. Кроме того, библиотека предоставляет удобный интерфейс для интеграции с другими библиотеками, такими как Keras и TensorFlow, что позволяет использовать уже готовые модели и переносить результаты между различными проектами.

Параметры:

- **num_classes** Целое число (int). По умолчанию None. Если None, то оно будет получено из данных.
- **multi_label** bool: Булев тип. По умолчанию установлено значение False.
- **loss** Loss-функция Keras. По умолчанию используется 'binary_crossentropy' или 'categorical_crossentropy' в зависимости от количества классов.
- **metrics** Необязательный параметр, может принимать список метрик Keras, в виде строки, функции или экземпляра метрики, список списков метрик или словарь с именованными метриками. По умолчанию используется метрика 'accuracy'..
- **project_name** str: Строка. Название AutoModel. По умолчанию установлено значение 'image_classifier'.
- **max_trials** int: Целое число. Максимальное количество разных моделей Keras, которые будут протестированы. Поиск может завершиться до достижения максимального количества проб. По умолчанию равен 100.
- **directory** Optional[Union[str, pathlib.Path]]: Это строка. Путь к директории для сохранения результатов поиска. По умолчанию значение None, что создает папку с именем AutoModel в текущей директории.
- **objective** str: String. Название метрики модели для минимизации или максимизации, например, 'val_accuracy'. По умолчанию устанавливается 'val_loss'.
- **tuner** Optional[Union[str, Type[autokeras.engine.tuner.AutoTuner]]]: Строка или подкласс AutoTuner. Если строка, то должна быть одной из 'greedy', 'bayesian', 'hyperband' или 'random'. Она также может быть подклассом AutoTuner. Если не указано, то используется тюнер, специфичный для задачи, который сначала оценивает наиболее часто используемые модели для задачи, прежде чем исследовать другие модели.
- **overwrite** bool: Булевое значение. По умолчанию False. Если False, загружает существующий проект с тем же именем, если такой найден. В противном случае перезаписывает проект.
- **seed** Optional[int]: Это целочисленный параметр, который устанавливает случайное начальное значение для генератора случайных чисел, используемого во время обучения модели. Это позволяет воспроизводить результаты обучения на разных машинах и улучшает воспроизводимость результатов.
- **max_model_size** Целое число. Максимальное количество скалярных параметров в модели. Модели, размер которых превышает это значение, отбрасываются.
- **kwargs**: Этот параметр относится к конструктору AutoModel и позволяет указать любые аргументы, поддерживаемые этим классом. Класс AutoModel является базовым классом для всех поддерживаемых моделей в AutoKeras, таких как ImageClassifier, TextClassifier, StructuredDataClassifier, и другие. Конкретный список аргументов, поддерживаемых AutoModel, зависит от конкретного класса модели, который используется. В общем, аргументы могут включать в себя параметры для слоев, оптимизаторов, регуляризации и других компонентов модели.

In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.4 MB/s eta 0:00:00


In [ ]:
import autokeras as ak

AutoKeras для многоклассовой классификации изображений на примере датасета с водителями

- Параметр *multi_label* отвечает за тип задачи мульти-лейбл классификации. Если multi_label=True, то задача считается мульти-лейбл классификацией, в которой каждый объект может иметь несколько меток (классов) из множества возможных меток. Если multi_label=False (по умолчанию), то каждый объект имеет только одну метку из возможных.
- *tuner* - алгоритм оптимизации (например, 'bayesian' или 'random');
 Он может принимать значения: 'greedy', 'bayesian', 'hyperband' или 'random'

**Эксперимент №1.**

max_trials=5

tuner='bayesian'

In [ ]:
# Обучение модели
# Определение EarlyStopping-коллбека
es_callback = EarlyStopping(monitor='val_loss', patience=4)
start_time = time.time()
clf = ak.ImageClassifier(overwrite=True, max_trials=5, num_classes=10, multi_label=False, loss="categorical_crossentropy", metrics=["accuracy"], 
                         project_name="my_project", directory="my_dir", seed=88, max_model_size=None, tuner='bayesian', objective ='val_accuracy')
# Обучаем 10  моделей
clf.fit(x_train, y_train, epochs=10, batch_size=64, callbacks=[es_callback])
end_time = time.time()
# Вывод времени обучения и точности модели
print("Total training time:", end_time - start_time)
print("Training accuracy:", clf.evaluate(x_train, y_train)[1])
print("Test accuracy:", clf.evaluate(x_val, y_val)[1])

Trial 5 Complete [00h 06m 15s]
val_accuracy: 0.1190476194024086

Best val_accuracy So Far: 0.26739928126335144
Total elapsed time: 00h 28m 59s
Epoch 1/10
45/45 [==============================] - 12s 181ms/step - loss: 2.6213 - accuracy: 0.1042
Epoch 2/10
45/45 [==============================] - 8s 182ms/step - loss: 2.4515 - accuracy: 0.1070
Epoch 3/10
45/45 [==============================] - 8s 187ms/step - loss: 2.3876 - accuracy: 0.1298
Epoch 4/10
45/45 [==============================] - 9s 190ms/step - loss: 2.3277 - accuracy: 0.1386
Epoch 5/10
45/45 [==============================] - 8s 187ms/step - loss: 2.3239 - accuracy: 0.1632
Epoch 6/10
45/45 [==============================] - 8s 182ms/step - loss: 2.3005 - accuracy: 0.1512
Epoch 7/10
45/45 [==============================] - 8s 183ms/step - loss: 2.2427 - accuracy: 0.1723
Epoch 8/10
45/45 [==============================] - 8s 181ms/step - loss: 2.2260 - accuracy: 0.1747
Epoch 9/10
45/45 [==============================] - 8s 1

Total training time: 1864.3755667209625
90/90 [==============================] - 11s 98ms/step - loss: 2.1155 - accuracy: 0.3235
Training accuracy: 0.32350876927375793
5/5 [==============================] - 3s 227ms/step - loss: 2.0885 - accuracy: 0.3667
Test accuracy: 0.36666667461395264


In [ ]:
# чтобы получить num_models лучших моделей:
best_models = clf.tuner.get_best_models(num_models=3)

In [ ]:
best_models[0].summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0     

In [ ]:
best_models[1].summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 64, 64, 3)        7         
 n)                                                              
                                                                 
 random_flip (RandomFlip)    (None, 64, 64, 3)         0         
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 resnet152v2 (Functional)    (None, None, None, 2048)  583316

In [ ]:
best_models[2].summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 efficientnetb0 (Functional)  (None, 2, 2, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 10)                12810 

In [ ]:
# Предсказываем на основании лучшей модели на проверочной выборке
predicted_y = clf.predict(x_val)
print(predicted_y)

5/5 [==============================] - 0s 86ms/step
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Оцениваем лучшую модель с использованием тестовых данных.
print(clf.evaluate(x_val, y_val))

5/5 [==============================] - 2s 97ms/step - loss: 2.0885 - accuracy: 0.3667
[2.0885496139526367, 0.36666667461395264]


Аккураси получилась низкой, для лучших результатов нужно сделать больше запусков, поменяв параметр max_trials

**Эксперимент №2.**

max_trials=15

tuner='bayesian'

Запустим алгоритм на бОльшем количестве запусков и посмотрим, позволит ли это нам улучшить точность на проверочной выборке

In [ ]:
# Обучение модели
# Определение EarlyStopping-коллбека
es_callback = EarlyStopping(monitor='val_loss', patience=4)
start_time = time.time()
clf_2 = ak.ImageClassifier(overwrite=True, max_trials=15, num_classes=10, multi_label=False, loss="categorical_crossentropy", metrics=["accuracy"], 
                         project_name="my_project", directory="my_dir", seed=44, max_model_size=None, tuner='bayesian', objective ='val_accuracy')
# Обучаем 10  моделей
clf_2.fit(x_train, y_train, epochs=10, batch_size=64, callbacks=[es_callback], validation_data=(x_val, y_val))
end_time = time.time()
# Вывод времени обучения и точности модели
print("Total training time:", end_time - start_time)
print("Training accuracy:", clf_2.evaluate(x_train, y_train)[1])
print("Test accuracy:", clf_2.evaluate(x_val, y_val)[1])

Trial 15 Complete [00h 12m 16s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.9200000166893005
Total elapsed time: 01h 07m 14s


Total training time: 4089.2154545783997
90/90 [==============================] - 17s 157ms/step - loss: 0.1570 - accuracy: 0.9849
Training accuracy: 0.9849122762680054
5/5 [==============================] - 3s 149ms/step - loss: 0.3645 - accuracy: 0.9200
Test accuracy: 0.9200000166893005


Да, эксперимент удался, точность повысилась до 92% на проверочной выборке. Посмотрим на архитектуру лучшей модели:

In [ ]:
model_2 = clf_2.export_model()
model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 resnet101v2 (Functional)    (None, None, None, 2048)  42626560  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 10)                1003530   
                                                             

**Эксперимент №3.**

max_trials=25

tuner=None

В случае, когда тюнер не указывается, применяется тюнер 'greedy'

'Greedy' тюнер является наиболее простым тюнером и используется по умолчанию. Он рассматривает небольшой набор моделей, начиная с наиболее простой, и последовательно увеличивает их сложность. После каждого шага тюнер выбирает лучшую модель и продолжает увеличивать ее сложность, пока не достигнет максимально заданной глубины. Это означает, что тюнеру не нужно проводить вычислительно дорогостоящие эксперименты с очень сложными моделями, что делает его очень эффективным и быстрым. Однако, в связи с этим, 'greedy' тюнер может не сходиться к глобальному оптимуму и может пропустить более сложные модели, которые могут давать лучшее качество на данных.

In [ ]:
# Обучение модели
# Определение EarlyStopping-коллбека
es_callback = EarlyStopping(monitor='val_loss', patience=4)
start_time = time.time()
clf_3 = ak.ImageClassifier(overwrite=True, max_trials=25, num_classes=10, multi_label=False, loss="categorical_crossentropy", metrics=["accuracy"], 
                         project_name="my_project", directory="my_dir", seed=44, max_model_size=None, objective ='val_accuracy')
# Обучаем 10  моделей
clf_3.fit(x_train, y_train, epochs=10, batch_size=64, callbacks=[es_callback], validation_data=(x_val, y_val))
end_time = time.time()
# Вывод времени обучения и точности модели
print("Total training time:", end_time - start_time) 
print("Training accuracy:", clf_3.evaluate(x_train, y_train)[1])
print("Test accuracy:", clf_3.evaluate(x_val, y_val)[1])

Trial 25 Complete [00h 00m 15s]
val_accuracy: 0.9933333396911621

Best val_accuracy So Far: 0.9933333396911621
Total elapsed time: 00h 48m 39s


Total training time: 2927.768089532852
90/90 [==============================] - 1s 7ms/step - loss: 0.0062 - accuracy: 1.0000
Training accuracy: 1.0
5/5 [==============================] - 0s 7ms/step - loss: 0.0640 - accuracy: 0.9933
Test accuracy: 0.9933333396911621


In [ ]:
model_3 = clf_3.export_model()
model_3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 64, 64, 3)        7         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0     

# AutoKeras для бинарной классификации текстов на основе датасета IMDB (Internet Movie Database) 

Это база данных, в которой хранится информация о фильмах, телешоу, актерах, кинокритиках и других связанных с кино сущностях. В рамках машинного обучения датасет IMDB относится к набору данных, содержащему рецензии на фильмы. Он содержит отзывы, оставленные пользователями сайта IMDB для фильмов в жанре «экшн» и «драма». Каждый отзыв помечен как положительный или отрицательный, и используется для задач классификации текста, например, для определения тональности текста.

В данном примере мы не будем указывать параметр tuner - таким образом используется настраиваемый для конкретной задачи tuner, который сначала оценивает наиболее часто используемые модели для задачи, а затем исследует другие модели.

In [ ]:
from sklearn.datasets import load_files
import tensorflow as tf
import autokeras as ak

In [ ]:
# загружаем датасет с рецензиями для бинарной классификации текстов
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

# set path to dataset
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # this film was just brilliant casting


(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


Важный момент - исли используется "тяжелая" предобученная модель, например, bert, AutoKeras сам уменьшает параметр batch_size, не прерывая обучение.

In [ ]:
# Запустим классификатор текстов, сделаем 5 моделей
clf = ak.TextClassifier(
    overwrite=True, max_trials=3, objective = 'val_accuracy'
)  
# Feed the text classifier with training data.
hist=clf.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), batch_size=64)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 3 Complete [00h 01m 18s]

Best val_accuracy So Far: 0.8925600051879883
Total elapsed time: 00h 04m 20s
Epoch 1/6
391/391 [==============================] - 39s 96ms/step - loss: 0.4758 - accuracy: 0.7436 - val_loss: 0.2895 - val_accuracy: 0.8788
Epoch 2/6
391/391 [==============================] - 9s 24ms/step - loss: 0.2526 - accuracy: 0.8991 - val_loss: 0.2675 - val_accuracy: 0.8912
Epoch 3/6
391/391 [==============================] - 9s 23ms/step - loss: 0.1790 - accuracy: 0.9337 - val_loss: 0.3141 - val_accuracy: 0.8808
Epoch 4/6
391/391 [==============================] - 9s 22ms/step - loss: 0.1292 - accuracy: 0.9537 - val_loss: 0.3830 - val_accuracy: 0.8719
Epoch 5/6
391/391 [==============================] - 9s 24ms/step - loss: 0.1074 - accuracy: 0.9607 - val_loss: 0.3541 - val_accuracy: 0.8897
Epoch 6/6
391/391 [==============================] - 9s 23ms/step - loss: 0.0835 - accuracy: 0.9695 - val_loss: 0.3910 - val_accuracy: 0.8846


782/782 [==============================] - 6s 7ms/step - loss: 0.3910 - accuracy: 0.8846
[0.3910045623779297, 0.8846399784088135]


In [ ]:
# Получаем лучшую модель
mod=clf.export_model()
mod.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 expand_last_dim (ExpandLast  (None, 1)                0         
 Dim)                                                            
                                                                 
 text_vectorization (TextVec  (None, 512)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 512, 64)           320064    
                                                                 
 dropout (Dropout)           (None, 512, 64)           0         
                                                                 
 conv1d (Conv1D)             (None, 508, 256)          82176 

# Keras Tuner

Keras Tuner - это библиотека, которая помогает выбрать оптимальные гиперпараметры для моделей, созданных с помощью Keras. Это позволяет автоматизировать процесс оптимизации гиперпараметров, что может существенно сократить время и усилия, затраченные на поиск лучшей модели.

Keras Tuner обеспечивает удобный интерфейс для определения гиперпараметров, исследуемых во время поиска, а также для выбора стратегии поиска. Библиотека поддерживает несколько стратегий поиска гиперпараметров, включая случайный поиск, поиск по сетке и поиск по градиентам.

Основные компоненты Keras Tuner:

* HyperModel - класс, который определяет архитектуру модели, которую нужно настроить. Обычно он наследуется от класса keras.Model.
* HyperParameters - класс, который определяет гиперпараметры, которые необходимо настроить. Он содержит методы для определения типов гиперпараметров, их диапазонов значений и других характеристик.
* Tuner - класс, который определяет стратегию поиска гиперпараметров. Он принимает на вход объект HyperModel и HyperParameters, а затем настраивает гиперпараметры, используя выбранную стратегию поиска. Результатом работы Tuner является оптимальный набор гиперпараметров.
* Oracle - класс, который определяет правила остановки поиска. Он определяет, когда поиск должен быть остановлен на основе заданных условий, таких как количество итераций или улучшение метрики.
* Callbacks - классы обратного вызова, которые можно использовать для мониторинга и сохранения состояния поиска.

kerastuner.tuners - это модуль библиотеки Keras Tuner, который содержит различные классы тюнеров для поиска оптимальных гиперпараметров моделей Keras.

Тюнеры Keras Tuner позволяют оптимизировать гиперпараметры модели, выбирая наиболее перспективные варианты из заданного диапазона значений. В этом процессе тюнер автоматически обучает и оценивает модели с разными комбинациями гиперпараметров и выбирает лучшую на основе заданной метрики.

Классы тюнеров в kerastuner.tuners включают в себя:

- RandomSearch - ищет лучшие гиперпараметры, выбирая их случайным образом из заданного диапазона значений.
- Hyperband - комбинация методов градиентной оптимизации и оптимизации гиперпараметров. Этот тюнер использует алгоритм Hyperband для пропуска наиболее обещающих моделей и оптимизации гиперпараметров только для них.
- BayesianOptimization - использует байесовскую оптимизацию для поиска лучших гиперпараметров.
- Sklearn - тюнер для моделей, совместимых со Scikit-Learn. Он использует случайный поиск или генетические алгоритмы для оптимизации гиперпараметров.

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import keras_tuner as kt

<ipython-input-32-d9e8d2ba1c76>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# Определение функции для построения модели
def model_builder(hp):
  model = Sequential()
  #Настраиваем параметры  у первого слоя Conv2D
  #hp_units_dense = hp.Int('units', min_value=32, max_value=512, step=32)
  hp_units_conv2D = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Conv2D(filters=hp_units_conv2D, kernel_size=3, activation='relu'))
  model.add(Flatten())
  model.add(Dense(10))

  # Настраиваем параметр learning rate для оптимайзера
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Определение гиперпараметров для настройки
tuner_search = RandomSearch(model_builder,
                            objective='val_accuracy',
                            max_trials=10,
                            directory='output',
                            project_name='drivers_keras_tuner')

# Запуск настройки гиперпараметров
tuner_search.search(x_train, y_train, epochs=10, batch_size=64,  validation_data=(x_val, y_val))

# Получение наилучшей модели
best_model = tuner_search.get_best_models(num_models=1)[0]

# Оценка наилучшей модели на тестовых данных
test_loss, test_accuracy = best_model.evaluate(x_val, y_val)
print('Test accuracy:', test_accuracy)

Trial 5 Complete [00h 03m 23s]
val_accuracy: 0.14666666090488434

Best val_accuracy So Far: 0.1599999964237213
Total elapsed time: 00h 24m 31s
5/5 [==============================] - 1s 221ms/step - loss: 8.7236 - accuracy: 0.1600
Test accuracy: 0.1599999964237213


In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 448)       12544     
                                                                 
 flatten (Flatten)           (None, 1722112)           0         
                                                                 
 dense (Dense)               (None, 10)                17221130  
                                                                 
Total params: 17,233,674
Trainable params: 17,233,674
Non-trainable params: 0
_________________________________________________________________


Точность получилась очень низкой. Попробуем добавить слоев в модель, изменим Tuner на Hyperband

In [ ]:
# Определение функции для построения модели
def model_builder_2(hp):
  model = Sequential()
  hp_units_dense = hp.Int('units', min_value=32, max_value=512, step=32)
  # Определение количества слоев и их размерности
  for i in range(hp.Int('num_layers', 1, 4)):
      model.add(Conv2D(filters=hp.Int('units_' + str(i), 32, 256, 32), kernel_size= 3, activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid'])))
  # Определение количества слоев и их размерности
  for i in range(hp.Int('num_layers', 1, 4)):
      model.add(Dense(units=hp.Int('units_' + str(i), 32, 256, 32), activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid'])))
  model.add(Flatten())
  model.add(Dense(10, activation="softmax"))
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4])

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Определение гиперпараметров для настройки
tuner_search = Hyperband(model_builder_2,
                            objective='val_accuracy', max_epochs=15,
                            
                            directory='output',
                            project_name='drivers_keras_tuner_2')

# Запуск настройки гиперпараметров
tuner_search.search(x_train, y_train, epochs=7, batch_size=64,  validation_data=(x_val, y_val))


Trial 30 Complete [00h 01m 47s]
val_accuracy: 0.9800000190734863

Best val_accuracy So Far: 0.9866666793823242
Total elapsed time: 00h 12m 56s


In [ ]:
models = tuner_search.get_best_models(num_models=2)


In [ ]:
best_model = tuner_search.get_best_models(num_models=1)[0]
best_model

In [ ]:
# Получение наилучшей модели
best_model_2 = tuner_search.get_best_models(num_models=1)[0]

# Оценка наилучшей модели на тестовых данных
test_loss, test_accuracy_2 = best_model_2.evaluate(x_val, y_val)
print('Test accuracy:', test_accuracy_2)

5/5 [==============================] - 1s 11ms/step - loss: 0.0667 - accuracy: 0.9867
Test accuracy: 0.9866666793823242


In [ ]:
best_model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        18464     
                                                                 
 dense (Dense)               (None, 60, 60, 64)        2112      
                                                                 
 dense_1 (Dense)             (None, 60, 60, 32)        2080      
                                                                 
 flatten (Flatten)           (None, 115200)            0         
                                                                 
 dense_2 (Dense)             (None, 10)                1152010   
                                                                 
Total params: 1,176,458
Trainable params: 1,176,458
Non-

KerasTuner включает несколько подклассов, называемых "tuners", которые определяют, какую стратегию использовать при поиске наилучших гиперпараметров. Некоторые из наиболее популярных тюнеров в KerasTuner включают GridSearch, RandomSearch и Hyperband.

RandomSearch выбирает случайную комбинацию гиперпараметров и проверяет ее. Hyperband - более эффективный тюнер, который использует несколько итераций и удаляет слабые модели, чтобы сосредоточиться на перспективных кандидатах.
Есть еще BayesianOptimization Tuner, Sklearn Tuner, The base Tuner class

После того как тюнер завершит процесс подбора наилучших гиперпараметров, он возвращает модель Keras, которая может быть использована для обучения на данных.

**Keras Tuner для бинарной классификации текстов**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Загрузка данных
data = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000)

# Преобразование данных
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=0, padding='post', maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=0, padding='post', maxlen=256)

# Определение функции для построения модели
def build_model(hp):
    model = keras.Sequential()
    # Добавление слоев эмбеддинга
    model.add(layers.Embedding(input_dim=10000, output_dim=hp.Int('embedding_dim', min_value=32, max_value=256, step=32)))
    model.add(layers.GlobalAveragePooling1D())
    # Определение количества слоев и их размерности
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 256, 32), activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid'])))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Определение оптимизатора и скомпилирование модели
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss=keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

# Определение гиперпараметров для настройки
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='imdb_sentiment_analysis')

# Запуск настройки гиперпараметров
tuner_search.search(train_data, train_labels, epochs=5, validation_split=0.2)

# Получение наилучшей модели
best_model = tuner_search.get_best_models(num_models=1)[0]

# Оценка наилучшей модели на тестовых данных
test_loss, test_accuracy = best_model.evaluate(test_data, test_labels)
print('Test accuracy:', test_accuracy)

Trial 5 Complete [00h 01m 37s]
val_accuracy: 0.8866000175476074

Best val_accuracy So Far: 0.8913999795913696
Total elapsed time: 00h 09m 55s
782/782 [==============================] - 3s 4ms/step - loss: 0.2996 - accuracy: 0.8806
Test accuracy: 0.8805999755859375


In [ ]:
# Получение наилучшей модели
best_model_3 = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
best_model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 192)         1920000   
                                                                 
 global_average_pooling1d (G  (None, 192)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 128)               24704     
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 128)               4224      
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                        

Функция build_model определяет архитектуру модели с помощью Keras Tuner. Она принимает объект HyperParameters, который содержит гиперпараметры, которые мы хотим оптимизировать. В этом примере мы оптимизируем гиперпараметры, связанные с размерностью векторного пространства эмбеддингов, количеством слоев и их размерностью, функциями активации и скоростью обучения.

Мы используем класс RandomSearch из Keras Tuner для настройки гиперпараметров. Метод search запускает процесс настройки, в котором модели обучаются на обучающем наборе данных и проверяются на валидационном наборе данных для оценки их производительности. Мы указываем максимальное количество испытаний (max_trials), которые хотим выполнить, а также папку directory и имя проекта project_name, в которых будут храниться результаты настройки.

После того, как настройка завершена, мы получаем наилучшую модель с помощью метода get_best_models. Затем мы оцениваем производительность этой модели на тестовом наборе данных с помощью метода evaluate.



# Talos

Talos - это библиотека для автоматической настройки гиперпараметров моделей глубокого обучения. Talos основана на библиотеке Keras и предоставляет простой и удобный API для оптимизации гиперпараметров.

С помощью Talos вы можете оптимизировать различные гиперпараметры, такие как размер батча, скорость обучения, количество слоев и их размерность, функции активации, параметры регуляризации и многое другое. Talos использует методы оптимизации, такие как Grid Search, Random Search, Fmin и другие, чтобы найти наилучшую комбинацию гиперпараметров.

Основной функциональностью Talos является Scan(), которая запускает процесс поиска оптимальных гиперпараметров. Она принимает на вход модель, гиперпараметры, настройки оптимизации и данные для обучения. После этого она запускает обучение и проверку модели с различными комбинациями гиперпараметров и сохраняет результаты.

In [ ]:
!pip install talos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 KB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 99.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 24.5 MB/s eta 0:00:00
  Created wheel for chances: filename=chances-0.1.9-py3-none-any.whl size=41609 sha256=e075b4aa67615cc9ecc94e3cdd19326e35be9d4c73a352d2ad3c3e58a9abc0d7
  Stored in directory: /root/.cache/pip/wheels/bc/47/dc/208d4038848287f804b6ededf43632ba7dbea530fdb3a069e4
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=637bd802b7ac238b941381c59927d7275f9706906a911a22971c34d294517758
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088de

In [ ]:
import talos as ta
from talos.model.normalizers import lr_normalizer
from talos import Scan
#from talos.utils import dataset

In [ ]:
# Определяем модель
def talos_model(x_train, y_train, x_val, y_val, params):
    model = Sequential()
    model.add(Conv2D(params['filters'], kernel_size=3, activation=params['activation'], input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(params['filters'], kernel_size=3, activation=params['activation']))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(params['hidden_units'], activation=params['activation']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(10, activation='softmax'))

    # Компилируем модель
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=params['learning_rate']), metrics=['accuracy'])

    # Обучаем модель
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        batch_size=params['batch_size'],
                        epochs=10,
                        verbose=0)
    return history, model

# Определяем гиперпараметры
p = {'filters': [16, 32],
     #'kernel_size': [(3, 3), (5, 5)],
     'hidden_units': [32, 64],
     'dropout': [0.2, 0.3],
     'batch_size': [32, 64],
     #'epochs': [5, 10],
     'activation': ['relu', 'elu'],
     'learning_rate': [0.001, 0.0001]}

# Запускаем оптимизацию гиперпараметров
scan_object = ta.Scan(x=x_train,
                      y=y_train,
                      x_val=x_val,
                      y_val=y_val,
                      model=talos_model,
                      params=p,
                      experiment_name='talos_mod')




100%|██████████| 64/64 [05:58<00:00,  5.60s/it]


In [ ]:
# получаем отчет о результатах сканирования
r = ta.Reporting(scan_object)

# выводим лучшие параметры
print(r.best_params)


<bound method Analyze.best_params of <talos.commands.analyze.Analyze object at 0x7f417cda3ac0>>


In [ ]:
r.data

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,filters,hidden_units,dropout,batch_size,activation,learning_rate
0,02/21/23-164042,02/21/23-164048,6.490367,10,0.314440,0.895088,0.218712,0.933333,16,32,0.2,32,relu,0.0010
1,02/21/23-164049,02/21/23-164054,5.077243,10,1.361121,0.556842,1.200270,0.640000,16,32,0.2,32,relu,0.0001
2,02/21/23-164054,02/21/23-164059,5.304686,10,0.057808,0.983509,0.126981,0.966667,16,32,0.2,32,elu,0.0010
3,02/21/23-164059,02/21/23-164104,4.714167,10,1.103133,0.682807,1.052074,0.666667,16,32,0.2,32,elu,0.0001
4,02/21/23-164104,02/21/23-164109,4.359731,10,0.253756,0.916140,0.180692,0.926667,16,32,0.2,64,relu,0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,02/21/23-164612,02/21/23-164618,6.012555,10,0.565079,0.855088,0.508742,0.886667,32,64,0.3,32,elu,0.0001
60,02/21/23-164618,02/21/23-164624,5.991820,10,0.282046,0.908772,0.164445,0.940000,32,64,0.3,64,relu,0.0010
61,02/21/23-164624,02/21/23-164630,5.990026,10,1.357684,0.546316,1.225586,0.620000,32,64,0.3,64,relu,0.0001
62,02/21/23-164631,02/21/23-164635,4.367067,10,0.031867,0.994035,0.067176,0.973333,32,64,0.3,64,elu,0.0010


In [ ]:
best_params = r.best_params(metric='val_accuracy', exclude=['param1', 'param2'])

In [ ]:
best_params

array([[32, 'elu', '02/21/23-164434', 0.07079509645700455, 0.3, 10,
        5.829704999923706, 32, 0.9845613837242126, 32,
        0.05771593749523163, 0.001, '02/21/23-164440', 0],
       [64, 'elu', '02/21/23-164631', 0.06717603653669357, 0.3, 10,
        4.367067098617554, 32, 0.9940350651741028, 64,
        0.031866975128650665, 0.001, '02/21/23-164635', 1],
       [32, 'elu', '02/21/23-164303', 0.09059999138116837, 0.3, 10,
        4.776015281677246, 16, 0.9933333396911621, 64,
        0.03410322219133377, 0.001, '02/21/23-164307', 2],
       [32, 'elu', '02/21/23-164519', 0.1406450718641281, 0.2, 10,
        5.997753858566284, 32, 0.9891228079795837, 64,
        0.03539183735847473, 0.001, '02/21/23-164525', 3],
       [32, 'elu', '02/21/23-164054', 0.1269809454679489, 0.2, 10,
        5.304685592651367, 16, 0.9835087656974792, 32,
        0.05780809000134468, 0.001, '02/21/23-164059', 4],
       [64, 'elu', '02/21/23-164158', 0.10445954650640488, 0.3, 10,
        3.5669081211090

# Auto-sklearn

Auto-sklearn - это библиотека автоматического машинного обучения на основе фреймворка scikit-learn. Она использует методики обучения на основе градиентного спуска и генетические алгоритмы для автоматического поиска лучших параметров модели. Auto-sklearn поддерживает как задачи классификации, так и задачи регрессии. Эта библиотека позволяет автоматически настраивать гиперпараметры и выбирать алгоритмы машинного обучения для данной задачи, что упрощает процесс обучения моделей и повышает их качество.

AutoSklearnClassifier является классификатором, основанным на автоматическом машинном обучении. Он использует методы оптимизации для автоматического подбора модели и гиперпараметров на основе заданной метрики качества.

Некоторые из важных параметров AutoSklearnClassifier:

- time_left_for_this_task: (int, default=3600) - ограничение времени, выделенное для автоматического поиска лучшей модели. Значение указывается в секундах.
- per_run_time_limit: (int, default=360) - максимальное время работы для одного вызова модели, выраженное в секундах.
- memory_limit: (int, default=3072) - ограничение памяти, которую может использовать модель в мегабайтах.
- ensemble_size: (int, default=50) - количество лучших моделей, выбранных для ансамблирования.
- resampling_strategy: (str, default='holdout') - стратегия для оценки качества моделей. Допустимые значения - holdout, cv, cv-10, cv-5x2, subsample, subsample-iterative-fit.
- metric: (str, default='accuracy') - метрика, используемая для оценки качества моделей. Допустимые значения - accuracy, balanced_accuracy, f1, roc_auc, precision, recall, log_loss.
- n_jobs: (int, default=1) - количество параллельных процессов для обучения моделей.

In [ ]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 KB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 KB 4.6 MB/s eta 0:00:00
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641945 sha256=d07638e59cabcc606e44b27f3cb5e638c4c5d746928c6a2bcd422074bced2b6

**Пример использования библиотеки auto-sklearn на основе табличных данных (датасет breast_cancer)**

In [ ]:
from pprint import pprint
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection


In [ ]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, random_state=1
)

In [ ]:
import autosklearn
from autosklearn import classification
cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_classification_example_tmp' 
)

In [ ]:
cls.fit(X_train, y_train, dataset_name="breast_cancer")

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      per_run_time_limit=30, time_left_for_this_task=120,
                      tmp_folder='/tmp/autosklearn_classification_example_tmp')

In [ ]:
print(cls.leaderboard())

          rank  ensemble_weight               type      cost  duration
model_id                                                              
7            2             0.10        extra_trees  0.014184  1.937602
27           1             0.10        extra_trees  0.014184  2.812684
16           4             0.10  gradient_boosting  0.021277  1.329778
21           3             0.04        extra_trees  0.021277  1.874241
2            5             0.04      random_forest  0.028369  2.195254
3           13             0.12                mlp  0.028369  1.327508
10          12             0.02      random_forest  0.028369  2.677490
11          11             0.02      random_forest  0.028369  3.557857
13           9             0.02  gradient_boosting  0.028369  1.792805
14          10             0.02                mlp  0.028369  2.617734
19           8             0.04        extra_trees  0.028369  4.960261
22           7             0.12  gradient_boosting  0.028369  1.425581
26    

In [ ]:
pprint(cls.show_models(), indent=4)

{   2: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7ff78ca32fa0>,
           'cost': 0.028368794326241176,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7ff78ca32d30>,
           'ensemble_weight': 0.04,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7ff78ca32160>,
           'model_id': 2,
           'rank': 1,
           'sklearn_classifier': RandomForestClassifier(max_features=5, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)},
    3: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7ff78c996160>,
           'cost': 0.028368794326241176,
           'data_preprocessor': <autosklearn.pipeline.component